# StyleGAN2 operations

### Preparation


**Run this cell after each session restart**

In [ ]:
#@title General setup { display-mode: "form", run: "auto" }

from IPython.display import HTML, Image, display
from base64 import b64encode
import ipywidgets as widgets
import os

%tensorflow_version 1.x
!apt-get -qq install ffmpeg
from google.colab import drive
drive.mount('/G', force_remount=True)
gdir = !ls /G/
gdir = '/G/%s/' % str(gdir[0])
%cd $gdir

#@markdown Copying StyleGAN2 to the directory below (on your Google drive):
work_dir = 'sg2_eps' #@param {type:"string"}
#@markdown NB: all paths below are relative to this directory, except archive with source images (it's relative to G drive root).

work_dir = gdir + work_dir + '/'
if not os.path.isdir(work_dir):
  !git clone -b colab git://github.com/eps696/stylegan2 $work_dir
%cd $work_dir
!pip install -r requirements.txt

from src.util.utilgan import file_list, basename
def makevid(seq_dir, size=None):
  out_sequence = seq_dir + '/%06d.jpg'
  out_video = seq_dir + '.mp4'
  !ffmpeg -y -v warning -i $out_sequence $out_video
  data_url = "data:video/mp4;base64," + b64encode(open(out_video,'rb').read()).decode()
  wh = '' if size is None else 'width=%d height=%d' % (size, size)
  return """<video %s controls><source src="%s" type="video/mp4"></video>""" % (wh, data_url)

model = model_pkl = ''
def model_select(work_dir):
  models = file_list(work_dir, 'pkl', subdir=True)
  models = [m.replace(work_dir, '') for m in models if not basename(m) in ['submit_config', 'vgg16_zhang_perceptual']]
  def on_change(change):
    global model, model_pkl
    if change['type'] == 'change' and change['name'] == 'value':
      model = change['new']
      model = os.path.splitext(model)[0] # filename without extension => load custom network
      model_pkl = model + '.pkl' # filename with extension => load original network
  model_select = widgets.Dropdown(options=models, description='Found models:', style={'description_width': 'initial'}, layout={'width': 'max-content'})
  display(model_select)
  model_select.observe(on_change)

# model_select(work_dir)
print('.. Done ..')

## Training

First, let's prepare the dataset. Upload zip-archive with images onto Google drive and type its path here. 

Ensure all your images have the same color channels (monochrome, RGB or RGBA).

If you work with patterns or shapes (rather than compostions), turn on `multicrop` to crop square fragments from bigger images (effectively increasing amount of data). They will be cut into `size`px fragments, overlapped with `step` shift by X and Y. If the image is smaller, it will be upscaled to the `size`. *Omit this, if you have edited the images yourself, e.g. to keep the compositions, or to work with non-square aspect ratios.*

In [ ]:
#@title Data setup 
dataset = 'test' #@param {type:"string"}
source = 'src.zip' #@param {type:"string"}
data_dir = os.path.join('/content', dataset)

#@markdown Multicrop
multicrop = True  #@param {type:"boolean"}
size = 512 #@param [256,512,1024]
overlap = 256 #@param {type:"integer"}
size = int(size)

# cleanup previous attempts
!rm -rf /content/tmp 
!rm -rf /content/*.tfr
!rm -rf $data_dir

!mkdir /content/tmp
%cd /content/tmp
fpath = os.path.join(gdir, source)
!unzip -o -q $fpath
%cd $work_dir

if multicrop:
  if os.path.isdir('/content/tmp'):
    %run src/util/multicrop.py --in_dir /content/tmp --out_dir $data_dir --size $size --step $overlap
  else:
    print('/content/tmp not found!!')
else:
  !mv /content/tmp $data_dir

%run src/training/dataset_tool.py --dataset $data_dir

data_dir = os.path.join(work_dir, 'data')
!mv /content/*.tfr $data_dir
!ls $data_dir/*.tfr

Now, we can train StyleGAN2 on the prepared dataset.

In [ ]:
#@title Train
%cd $work_dir
dataset = 'test' #@param {type:"string"}
lod_step_kimg = 30 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --dataset $data_dir --lod_step_kimg $lod_step_kimg --d_aug

> This will run training process, according to the options in `src/train.py`. Results (models and samples) are saved under `train` directory, similar to original Nvidia approach. There are two types of models saved: compact (containing only Gs network for inference) as `<dataset>-...pkl` (e.g. `mydata-512-0360.pkl`), and full (containing G/D/Gs networks for further training) as `snapshot-...pkl`. Check sample images there to follow the progress.

> By default, the most powerful SG2 config (F) is used; if you face OOM issue, you may resort to `--config E`, requiring less memory (with poorer results, of course). [Differential Augmentation](https://github.com/mit-han-lab/data-efficient-gans) mode `--d_aug` is turned on for more effective training (absolutely required for small datasets with ~100x images). 

> The length of the training is defined by `--lod_step_kimg X` argument. It's kind of legacy from [progressive GAN](https://github.com/tkarras/progressive_growing_of_gans) and defines one step of progressive training. Network with base resolution 1024px will be trained for 20 such steps, for 512px - 18 steps, et cetera. Reasonable `lod_step_kimg` value for full training on big datasets is 300-600, while in `--d_aug` mode 20-40 is sufficient.

Other training options:

In [ ]:
%run src/train.py --help

If the training process was interrupted, resume it from the last saved model.

In [ ]:
#@title Resume training
%cd $work_dir
dataset = 'test' #@param {type:"string"}
resume_dir = 'train/000-test-512-f-daug' #@param {type:"string"}
lod_step_kimg = 30 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --dataset $data_dir --resume $resume_dir --d_aug --lod_step_kimg $lod_step_kimg 

NB: In most cases it's much easier to use a "transfer learning" trick, rather than perform full training from the scratch. For that, we use existing well-trained model as a starter, and "finetune" (uptrain) it with our data. This works pretty well, even if our dataset is very different from the original model. 

So here is a faster way to train our GAN (presuming we have full trained model `train/ffhq-512.pkl` already):

In [ ]:
#@title Finetune training
%cd $work_dir
dataset = 'test' #@param {type:"string"}
resume_pkl = 'train/ffhq-512.pkl' #@param {type:"string"}
lod_step_kimg = 30 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --dataset $data_dir --resume $resume_pkl --d_aug --lod_step_kimg $lod_step_kimg --finetune

There's no need to go for exact steps, you may stop when you're ok with the results. Lower `lod_step_kimg` helps following the progress.

## Generation

Let's produce some imagery from the original cat model (get it [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl) and put onto Google drive within our working directory).  
Generated results are saved as sequences and videos (by default, under `_out` directory).  
More cool models can be found [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2).

In [ ]:
#@title ### Generator setup
!cd $work_dir

output = '_out/cats' #@param {type:"string"}

frames = 50 #@param {type:"integer"}
frame_step = 10 #@param {type:"integer"}
timeframe = '%d-%d' % (frames, frame_step)

cubic_smooth = True #@param {type:"boolean"}
gauss_smooth = False #@param {type:"boolean"}
cubic_smooth = '--cubic ' if cubic_smooth else ''
gauss_smooth = '--gauss ' if gauss_smooth else ''
smooth = cubic_smooth + gauss_smooth

save_lat = False #@param {type:"boolean"}
save_lat = '--save_lat ' if save_lat else ''

#@markdown Select model from the dropdown below:
model_select(work_dir)

> This means loading the model, and producing 50 frames, interpolating between random latent space keypoints, with a step of 10 frames between keypoints. 
`save_lat` option = save all traversed dlatent points as Numpy array in `*.npy` file (useful for further curating). `cubic` option changes linear interpolation to cubic for smoother animation; `gauss` provides additional smoothing.

In [ ]:
#@title ### Native generation
%cd $work_dir
%run src/_genSGAN2.py --model $model_pkl --out_dir $output --frames $timeframe $smooth $save_lat
HTML(makevid(output))

In [ ]:
#@title ### Custom size generation

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size  $smooth $save_lat
HTML(makevid(output))

In [ ]:
#@title ### Multi-latent  generation

sizeX = 768 #@param {type:"integer"} 
sizeY = 256 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

split_X =  3#@param {type:"integer"} 
split_Y =  1#@param {type:"integer"}
split = '%d-%d' % (split_X, split_Y)
splitfine = 0. #@param {type:"number"}

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size -n $split --splitfine $splitfine $smooth
HTML(makevid(output))

> Here we get animated composition of 3 independent frames, blended together horizontally.  
`splitfine` controls boundary fineness (0 = smoothest/default, higher => thinner).  

Instead of frame splitting, we can load external mask from b/w image file (or folder with image sequence):

In [ ]:
#@title ### Masked  generation

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

lat_mask = '_in/mask.jpg' #@param {type:"string"} 

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --latmask $lat_mask  $smooth
HTML(makevid(output))

`digress` adds some funky displacements by tweaking initial constant layer.  
`truncation` controls variety, as usual  (0 = boring, >1 = weird). 

In [ ]:
#@title ### Other tricks

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

digress =  2#@param {type:"number"} 
truncation =  1.5#@param {type:"number"} 

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --digress $digress --trunc $truncation $smooth
HTML(makevid(output))

## Latent space exploration

For these experiments download [FFHQ model](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl) and save to `models` directory.

In [ ]:
#@title ### Generator setup
!cd $work_dir

output = '_out/ffhq' #@param {type:"string"}

frames = 50 #@param {type:"integer"}
frame_step = 10 #@param {type:"integer"}
timeframe = '%d-%d' % (frames, frame_step)

cubic_smooth = True #@param {type:"boolean"}
gauss_smooth = False #@param {type:"boolean"}
cubic_smooth = '--cubic ' if cubic_smooth else ''
gauss_smooth = '--gauss ' if gauss_smooth else ''
smooth = cubic_smooth + gauss_smooth

save_lat = False #@param {type:"boolean"}
save_lat = '--save_lat ' if save_lat else ''

#@markdown Select model from the dropdown list below:
model_select(work_dir)

Project external images onto the model dlatent space, saving points as Numpy arrays in `*.npy` files.  
Hint: download [VGG model](https://drive.google.com/uc?id=1N2-m9qszOeVC9Tq77WxsLnuWwOedQiD2) and save it as `models/vgg/vgg16_zhang_perceptual.pkl` to avoid auto-downloading errors.

In [ ]:
#@title ### Images projection

image_dir = '_in/photo' #@param {type:"string"} 
out_dir = '_out/proj' #@param {type:"string"} 
steps =  100#@param {type:"integer"} 

%cd $work_dir
%run src/project_latent.py --model $model_pkl --in_dir _in/photo --out_dir _out/proj --steps $steps

from src.util.utilgan import img_list
images = img_list(out_dir, subdir=True)
images = [f for f in images if '-%04d.jpg' % steps in f]
Image(images[0], width=512, height=512)

Produce looped animation, interpolating between saved dlatent points (from a file or directory with `*.npy` or `*.npz` files). To select only few frames from a sequence `xxx.npy`, create text file with comma-delimited frame numbers and save it as `xxx.txt` in the same directory (check examples in `_in/dlats`). 

In [ ]:
#@title ### "Walk" between saved dlatent points

dlatents = '_in/dlats' #@param {type:"string"} 
out_dir = '_out/ffhq-dlats' #@param {type:"string"} 

%cd $work_dir
%run src/_play_dlatents.py --model $model --dlatents $dlatents --out_dir $out_dir --fstep $frame_step $smooth
HTML(makevid(out_dir, size=512))



We can also load another dlatent point and apply it to higher network layers, effectively "stylizing" images with its' fine features.  
`digress` and `truncation` are also applicable here.

In [ ]:
#@title ### "Walk" between saved points w/tricks

dlatents = '_in/dlats' #@param {type:"string"} 
out_dir = '_out/ffhq-dlats' #@param {type:"string"} 
style_dlat = '_in/blonde458.npy' #@param {type:"string"} 
digress =  2#@param {type:"number"} 
truncation =  1.5#@param {type:"number"} 

%cd $work_dir
%run src/_play_dlatents.py --model $model --dlatents $dlatents --out_dir $out_dir --fstep $frame_step $smooth --style_npy_file $style_dlat --digress $digress --trunc $truncation
HTML(makevid(out_dir, size=512))

Generate animation by walking from saved dlatent point along feature direction vectors (aging/smiling/etc) one by one.

In [ ]:
#@title ### "Walk" along feature directions

main_dlat = '_in/blonde458.npy' #@param {type:"string"} 
vectors = '_in/vectors_ffhq' #@param {type:"string"} 
out_dir = '_out/ffhq_looks' #@param {type:"string"} 

%cd $work_dir
%run src/_play_vectors.py --model $model_pkl --npy_file $main_dlat --vector_dir $vectors --out_dir $out_dir --fstep $frame_step
HTML(makevid(out_dir, size=512))

## Tweaking models

NB: No real examples here! The commands are for reference, try with your own files.  
All paths are relative to StyleGAN2 copy on G drive.

Strip G/D networks from a full model, leaving only Gs for inference. Resulting file is saved with `-Gs` suffix.  
It's recommended to reconstruct the network, saving necessary arguments with it. Useful for foreign downloaded models.

In [ ]:
model = 'snapshot-1024.pkl' #@param {type:"string"} 
reconstruct = True #@param {type:"boolean"} 
reconstruct = '-r ' if reconstruct else ''
%cd $work_dir
%run src/model_convert.py --source $model $reconstruct

Add or remove layers (from a trained model) to adjust its resolution for further finetuning. For given example, this would produce new model with 512px resolution, populating weights on the layers up to 256px from the source snapshot; the rest will be initialized randomly. It also can decrease resolution (say, make 512 from 1024). Note that this effectively changes the number of layers in the model.   
This option works with complete (G/D/Gs) models only, since its' purpose is transfer-learning (the resulting model will contain either partially random weights, or wrong `ToRGB` params). 

In [ ]:
model = 'snapshot-256.pkl' #@param {type:"string"}
resolution = 512 #@param [256, 512, 1024]
resolution = int(resolution)

%cd $work_dir
%run src/model_convert.py --source $model --res 512

Crop resolution of a trained model. This will produce working non-square model. Opposite to the method above, this one doesn't change layer count. This is experimental feature (as stated by the author @Aydao), also using some voluntary logic, so works only with basic resolutions.

In [ ]:
model = 'snapshot-1024.pkl' #@param {type:"string"} 
resX = 1024 #@param [256, 384, 512, 768, 1024]
resX = int(resX)
resY = 768 #@param [256, 384, 512, 768, 1024]
resY = int(resY)
res = '%d-%d' % (resX, resY)

%cd $work_dir
%run src/model_convert.py --source $model --res $res

Combine lower layers (pose/shape) from one model with higher layers (style/texture) from another. The models are switched at the layer with size `resolution` (usually 32/64/128); `level` is 0 or 1.  
This method works properly only for models from one "family", i.e. uptrained (finetuned) from the same original model. 

In [ ]:
model1 = 'model1.pkl' #@param {type:"string"}
model2 = 'model2.pkl' #@param {type:"string"}
resolution = 32 #@param [32, 64, 128]
resolution = int(resolution)
level = 0 #@param [0, 1]
level = int(level)

%cd $work_dir
%run src/models_blend.py --pkl1 $model1 --pkl2 $model2 --res $resolution --level $level

Mix few models by stochastic averaging all weights.  
This method also works properly only for models from one "family", i.e. uptrained (finetuned) from the same original model. 

In [ ]:
models_dir = 'models' #@param {type:"string"}

%cd $work_dir
%run src/models_swa.py --in_dir $models_dir